In [1]:
# https://github.com/andywu0913/OpenAI-GYM-CarRacing-DQN/tree/master - Testing functin inspired by the training function in this repo
# https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

### Imports

In [2]:
import gym
from collections import deque

import pandas as pd
import torch

from agent import Agent
from processing import process_state_image, generate_state_frame_stack_from_queue

In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Variables

In [4]:
# Set the folder path of the model you want to test
folder = "runs/run_20240704234303"

In [5]:
# Read config file to get the configurations
config_df = pd.read_csv(folder + '/configurations.csv')
configurations = pd.Series(config_df.Value.values, index=config_df.Parameter).to_dict()

In [6]:
# Setting up all variables needed for initialization of the agent and the testing
seed = int(configurations['seed'])
episodes = int(configurations['episodes'])
frame_stack_num = int(configurations['frame_stack_num'])
memory_size = int(configurations['memory_size'])
gamma = configurations['gamma']
epsilon = configurations['epsilon']
epsilon_min = configurations['epsilon_min']
epsilon_decay = configurations['epsilon_decay']
learning_rate = float(configurations['learning_rate'])
batch_size = int(configurations['batch_size'])
#update_every_n = int(configurations['update_every_n'])
#negative_reward_counter_n = int(configurations['negative_reward_counter_n'])

### Environment and Agent

In [7]:
# Create the environment and the agent
env = gym.make(
    'CarRacing-v2',
    render_mode='rgb_array',
    continuous=False,
    max_episode_steps=1000
)

testing_agent = Agent(
    action_space=env.action_space,
    frame_stack_num=frame_stack_num,
    memory_size=memory_size,
    gamma=gamma,
    epsilon=epsilon,
    epsilon_min=epsilon_min,
    epsilon_decay=epsilon_decay,
    learning_rate=learning_rate,
    seed=seed
)

# Load the trained model
testing_agent.load(f'{folder}/trained_model.pth')

### Testing

In [8]:
def testing(env, test, seed):
    """
    Function to test the model.

    Args:
        env (gym.Env): The gym environment to be tested.
        test (Agent): The testing agent.
        seed (int): Seed for the environment.

    Returns:
        float: The total reward obtained during the test.
    """

    # Resetting the environment and setting up the video recording
    state, _ = env.reset(seed=seed)
    env = gym.wrappers.RecordVideo(
        env=env,
        video_folder=f'{folder}/Video',
        video_length=0
    )

    # Setting up the initial state and initializing the reward
    init_state = process_state_image(state)
    state_frame_stack_queue = deque([init_state] * test.frame_stack_num, maxlen=test.frame_stack_num)
    total_reward = 0

    # Playing the game
    done = False
    while not done:

        # Getting the action from the agent
        current_state_frame_stack = generate_state_frame_stack_from_queue(state_frame_stack_queue)
        action = test.act(current_state_frame_stack, testing=True)

        # Taking the action and getting the next state
        reward = 0
        for _ in range(frame_stack_num-1):
            next_state, r, terminated, truncated, info = env.step(action)
            if terminated or truncated:
                done = True
            else:
                done = False
            reward += r
            if done:
                break
        
        # Updating the total reward
        total_reward += reward

        # Adding the next state to the queue
        next_state = process_state_image(next_state)
        state_frame_stack_queue.append(next_state)
    
    # Closing the environment and returning the total reward
    env.close()
    return total_reward

In [9]:
testing(env, testing_agent, seed)

c:\Users\fabia\anaconda3\envs\car\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at c:\Users\fabia\Desktop\Studium\RL\RL_CarRacing\Training\runs\run_20240704234303\Video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video c:\Users\fabia\Desktop\Studium\RL\RL_CarRacing\Training\runs\run_20240704234303\Video\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\fabia\Desktop\Studium\RL\RL_CarRacing\Training\runs\run_20240704234303\Video\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\fabia\Desktop\Studium\RL\RL_CarRacing\Training\runs\run_20240704234303\Video\rl-video-episode-0.mp4


699.9999999999839